In [1]:
import os
import time
import numpy as np
import pybullet as p
import pybullet_data
import cv2
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from scipy.linalg import pinv
from scipy.spatial.transform import Rotation as Rot
import torch
import clip
from PIL import Image

pybullet build time: Nov 28 2023 23:45:17


In [2]:
# CLIP SETUP
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/32", device=device)

def encode_text(text):
    text_tokens = clip.tokenize([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

def encode_image(pil_img):
    image_input = preprocess(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features

cuda


In [3]:
#camera (don't change these settings)
camera_width = 512                                             #image width
camera_height = 512                                            #image height
camera_fov = 120                                                #field of view of camera
camera_focal_depth = 0.5*camera_height/np.tan(0.5*np.pi/180*camera_fov) 
                                                               #focal depth in pixel space
camera_aspect = camera_width/camera_height                     #aspect ratio
camera_near = 0.02                                             #near clipping plane in meters, do not set non-zero
camera_far = 100                                               #far clipping plane in meters


#control objectives (if you wish, you can play with these values for fun)
object_location_desired = np.array([camera_width/2,camera_height/2])
                                                               #center the object to middle of image 
K_p_x = 0.1                                                    #Proportional control gain for translation
K_p_Omega = 0.02                                               #Proportional control gain for rotation  

# Robot with Camera Class
class eye_in_hand_robot:
    def get_ee_position(self):
        '''
        Function to return the end-effector of the link. This is the very tip of the robot at the end of the jaws.
        '''
        endEffectorIndex = self.numActiveJoints
        endEffectorState = p.getLinkState(self.robot_id, endEffectorIndex)
        endEffectorPos = np.array(endEffectorState[0])
        endEffectorOrn = np.array(p.getMatrixFromQuaternion(endEffectorState[1])).reshape(3,3)
        
        #add an offset to get past the forceps
        endEffectorPos += self.camera_offset*endEffectorOrn[:,2]
        return endEffectorPos, endEffectorOrn

    def get_current_joint_angles(self):
        # Get the current joint angles
        joint_angles = np.zeros(self.numActiveJoints)
        for i in range(self.numActiveJoints):
            joint_state = p.getJointState(self.robot_id, self._active_joint_indices[i])
            joint_angles[i] = joint_state[0]
        return joint_angles
    
    def get_jacobian_at_current_position(self):
        #Returns the Robot Jacobian of the last active link
        mpos, mvel, mtorq = self.get_active_joint_states()   
        zero_vec = [0.0]*len(mpos)
        linearJacobian, angularJacobian = p.calculateJacobian(self.robot_id, 
                                                              self.numActiveJoints,
                                                              [0,0,self.camera_offset],
                                                              mpos, 
                                                              zero_vec,
                                                              zero_vec)
        #only return the active joint's jacobians
        Jacobian = np.vstack((linearJacobian,angularJacobian))
        return Jacobian[:,:self.numActiveJoints]
    
    def set_joint_position(self, desireJointPositions, kp=1.0, kv=0.3):
        '''Set  the joint angle positions of the robot'''
        zero_vec = [0.0] * self._numLinkJoints
        allJointPositionObjectives = [0.0]*self._numLinkJoints
        for i in range(desireJointPositions.shape[0]):
            idx = self._active_joint_indices[i]
            allJointPositionObjectives[idx] = desireJointPositions[i]

        p.setJointMotorControlArray(self.robot_id,
                                    range(self._numLinkJoints),
                                    p.POSITION_CONTROL,
                                    targetPositions=allJointPositionObjectives,
                                    targetVelocities=zero_vec,
                                    positionGains=[kp] * self._numLinkJoints,
                                    velocityGains=[kv] * self._numLinkJoints)

    def get_active_joint_states(self):
        '''Get the states (position, velocity, and torques) of the active joints of the robot
        '''
        joint_states = p.getJointStates(self.robot_id, range(self._numLinkJoints))
        joint_infos = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        joint_states = [j for j, i in zip(joint_states, joint_infos) if i[3] > -1]
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        joint_torques = [state[3] for state in joint_states]
        return joint_positions, joint_velocities, joint_torques


         
    def __init__(self, robot_id, initialJointPos):
        self.robot_id = robot_id
        self.eeFrameId = []
        self.camera_offset = 0.1 #offset camera in z direction to avoid grippers
        # Get the joint info
        self._numLinkJoints = p.getNumJoints(self.robot_id) #includes passive joint
        jointInfo = [p.getJointInfo(self.robot_id, i) for i in range(self._numLinkJoints)]
        
        # Get joint locations (some joints are passive)
        self._active_joint_indices = []
        for i in range(self._numLinkJoints):
            if jointInfo[i][2]==p.JOINT_REVOLUTE:
                self._active_joint_indices.append(jointInfo[i][0])
        self.numActiveJoints = len(self._active_joint_indices) #exact number of active joints

        #reset joints
        for i in range(self._numLinkJoints):
            p.resetJointState(self.robot_id,i,initialJointPos[i])


def draw_coordinate_frame(position, orientation, length, frameId = []):
    '''
    Draws a coordinate frame x,y,z with scaled lengths on the axes 
    in a position and orientation relative to the world coordinate frame
    pos: 3-element numpy array
    orientation: 3x3 numpy matrix
    length: length of the plotted x,y,z axes
    frameId: a unique ID for the frame. If this supplied, then it will erase the previous location of the frame
    
    returns the frameId
    '''
    if len(frameId)!=0:
        p.removeUserDebugItem(frameId[0])
        p.removeUserDebugItem(frameId[1])
        p.removeUserDebugItem(frameId[2])
    
    lineIdx=p.addUserDebugLine(position, position + np.dot(orientation, [length, 0, 0]), [1, 0, 0])  # x-axis in red
    lineIdy=p.addUserDebugLine(position, position + np.dot(orientation, [0, length, 0]), [0, 1, 0])  # y-axis in green
    lineIdz=p.addUserDebugLine(position, position + np.dot(orientation, [0, 0, length]), [0, 0, 1])  # z-axis in blue

    return lineIdx,lineIdy,lineIdz

def opengl_plot_world_to_pixelspace(pt_in_3D_to_project, viewMat, projMat, imgWidth, imgHeight):
    ''' Plots a x,y,z location in the world in an openCV image
    This is used for debugging, e.g. given a known location in the world, verify it appears in the camera
    when using p.getCameraImage(...). The output [u,v], when plot with opencv, should line up with object 
    in the image from p.getCameraImage(...)
    '''
    pt_in_3D_to_project = np.append(pt_in_3D_to_project,1)
    #print('Point in 3D to project:', pt_in_3D_to_project)

    pt_in_3D_in_camera_frame = viewMat @ pt_in_3D_to_project
    #print('Point in camera space: ', pt_in_3D_in_camera_frame)

    # Convert coordinates to get normalized device coordinates (before rescale)
    uvzw = projMat @ pt_in_3D_in_camera_frame
    #print('after projection: ', uvzw)

    # scale to get the normalized device coordinates
    uvzw_NDC = uvzw/uvzw[3]
    #print('after normalization: ', uvzw_NDC)

    #x,y specifies lower left corner of viewport rectangle, in pixels. initial value is (0,)
    u = ((uvzw_NDC[0] + 1) / 2.0) * imgWidth
    v = ((1-uvzw_NDC[1]) / 2.0) * imgHeight

    return [int(u),int(v)]

    
def get_camera_view_and_projection_opencv(cameraPos, camereaOrn):
    '''Gets the view and projection matrix for a camera at position (3) and orientation (3x3)'''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    _, _, rgbImg, depthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #returns camera view and projection matrices in a form that fits openCV
    viewMat = np.array(__camera_view_matrix_opengl).reshape(4,4).T
    projMat = np.array(__camera_projection_matrix_opengl).reshape(4,4).T
    return viewMat, projMat

def get_camera_img_float(cameraPos, camereaOrn):
    ''' Gets the image and depth map from a camera at a position cameraPos (3) and cameraOrn (3x3) in space. '''
    __camera_view_matrix_opengl = p.computeViewMatrix(cameraEyePosition=cameraPos,
                                                   cameraTargetPosition=cameraPos+camereaOrn[:,2],
                                                   cameraUpVector=-camereaOrn[:,1])

    __camera_projection_matrix_opengl = p.computeProjectionMatrixFOV(camera_fov, camera_aspect, camera_near, camera_far)        
    width, height, rgbImg, nonlinDepthImg, _ = p.getCameraImage(camera_width, 
                                                 camera_height, 
                                                 __camera_view_matrix_opengl,
                                                 __camera_projection_matrix_opengl, 
                                                 renderer=p.ER_BULLET_HARDWARE_OPENGL)

    #adjust for clipping and nonlinear distance i.e., 1/d (0 is closest, i.e., near, 1 is furthest away, i.e., far
    depthImgLinearized =camera_far*camera_near/(camera_far+camera_near-(camera_far-camera_near)*nonlinDepthImg)

    #convert to numpy and a rgb-d image
    rgb_image = np.array(rgbImg[:,:,:3], dtype=np.uint8)
    depth_image = np.array(depthImgLinearized, dtype=np.float32)
    return rgb_image, depth_image

In [4]:
def getImageJacobian(u_px, v_px, depthImg, focal_length, imgWidth, imgHeight):
    """Calculate the image Jacobian (interaction matrix) for visual servoing."""

    f = focal_length
    z = depthImg[u_px,v_px]

    u = u_px - imgWidth/2
    v = v_px - imgHeight/2

    L = np.array([
        [-f/z, 0 ,-u/z, u*v/f, f+u**2/f,-v],
        [0,f/z, -v/z, f+v**2/f, u*v/f, u]
    ])

    return L

In [5]:
def findCameraControl(object_loc_des, object_loc, image_jacobian):
    """Calculate camera velocity screw with different gains for translation and rotation"""
    error = object_loc_des - np.array(object_loc)
    
    J_pinv = pinv(image_jacobian)
    raw_velocity = np.dot(J_pinv, error)
    
    # Apply different gains for translation and rotation
    velocity_screw = np.zeros(6)
    velocity_screw[:3] = 0.1 * raw_velocity[:3]     # Kp = 0.1 for translation
    velocity_screw[3:] = 0.02 * raw_velocity[3:]    # Kω = 0.02 for rotation
    
    delta_X = velocity_screw[:3]
    delta_Omega = velocity_screw[3:]
    
    return delta_X, delta_Omega

In [6]:
def findJointControl(robot, delta_X, delta_Omega):
    """
    Calculate joint velocities using the robot Jacobian and null space optimization
    
    Parameters:
        robot: robot instance containing Jacobian information
        delta_X: desired camera linear velocity in world frame
        delta_Omega: desired camera angular velocity in world frame
    """
    
    J_robot = robot.get_jacobian_at_current_position()
    
    V_desired = np.concatenate([delta_X, delta_Omega])
    
    damping = 0.01
    J_dag = J_robot.T @ np.linalg.inv(J_robot @ J_robot.T + damping**2 * np.eye(6))
    
    null_proj = np.eye(robot.numActiveJoints) - J_dag @ J_robot
    
    q_current = robot.get_current_joint_angles()
    
    q_comfortable = np.array([0, -np.pi/4, 0, -np.pi/2, 0, np.pi/2, np.pi/4])
    
    k0 = 0.1  

    delta_q = J_dag @ V_desired + k0 * null_proj @ (q_comfortable - q_current)
    
    return delta_q

In [7]:
def generate_candidate_boxes(img, num_boxes_horizontal=8, num_boxes_vertical=8):
    """
    Generate candidate bounding boxes by dividing the image into a grid.
    Each cell in the grid is considered a candidate bounding box.

    Parameters:
        img (np.array): The image as a numpy array (H, W, 3).
        num_boxes_horizontal (int): Number of candidate boxes along the width.
        num_boxes_vertical (int): Number of candidate boxes along the height.

    Returns:
        List[Tuple]: A list of candidate boxes in the form (x1, y1, x2, y2).
    """
    h, w, _ = img.shape
    box_width = w // num_boxes_horizontal
    box_height = h // num_boxes_vertical

    boxes = []
    for i in range(num_boxes_vertical):
        for j in range(num_boxes_horizontal):
            x1 = j * box_width
            y1 = i * box_height
            x2 = x1 + box_width
            y2 = y1 + box_height
            boxes.append((x1, y1, x2, y2))
    return boxes

def get_best_box(rgb, candidate_boxes, text_features, model, preprocess, device):
    # Extract all candidate crops
    crops = []
    for box in candidate_boxes:
        (x1, y1, x2, y2) = box
        pil_crop = Image.fromarray(rgb[y1:y2, x1:x2])
        crops.append(preprocess(pil_crop).unsqueeze(0))
    
    # Stack all crops into a single batch tensor
    batch = torch.cat(crops, dim=0).to(device)

    # Run the model once on the whole batch
    with torch.no_grad():
        image_features = model.encode_image(batch)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    # Compute similarities
    similarities = (image_features @ text_features.T).squeeze()

    # Find best matching box
    best_idx = similarities.argmax().item()
    best_box = candidate_boxes[best_idx]
    best_similarity = similarities[best_idx].item()
    return best_box, best_similarity


In [8]:
import matplotlib.pyplot as plt
def plot_final_results(similarity):
    """Create final summary plots of roll angle and tracking error"""
    plt.figure(figsize=(12, 8))
    
    # Roll angle subplot
    
    plt.plot(similarity, label='Similarity Change')
    plt.title('Similarity Change')
    plt.ylabel('Similarity')
    plt.xlabel('Time Step')
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [17]:
from scipy.spatial.transform import Rotation as R
# =====================
# PyBullet Setup
p.connect(p.GUI)
time_step = 0.001
p.resetSimulation()
p.setTimeStep(time_step)
p.setGravity(0, 0, -9.8)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF('plane.urdf')



# Load Panda
pandaUid = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)
initialJointPosition = [0, -np.pi/4, np.pi/4, -np.pi/4, np.pi/4, np.pi/4, np.pi/4, 0, 0, 0, 0, 0]
robot = eye_in_hand_robot(pandaUid, initialJointPosition)
# p.stepSimulation()

angle = np.radians(45)
forward = np.array([np.cos(angle), np.sin(angle), 0])
up = np.array([0,0,-1])
right = np.cross(up, forward)
cameraOrientation = np.column_stack((right, up, forward))
cameraPosition = np.array([0,0,1])
# Convert rotation matrix to quaternion using SciPy
rot = R.from_matrix(cameraOrientation)
target_quat = rot.as_quat()  # [x, y, z, w]

endEffectorIndex = robot.numActiveJoints
ik_solution = p.calculateInverseKinematics(pandaUid, endEffectorIndex, cameraPosition, target_quat)

# Set joints to IK solution
for i, joint_index in enumerate(robot._active_joint_indices):
    p.resetJointState(pandaUid, joint_index, ik_solution[i])

p.stepSimulation()

# Now camera is at the EE with desired orientation/position
cameraPosition, cameraOrientation = robot.get_ee_position()
print("Initial EE pose:", cameraPosition, cameraOrientation)

# Place a box
box_length, box_width, box_depth = 0.4, 0.4, 0.4
target_box_center = [1.5, 0.5, 0.3]
geomBox = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2])
visualBox = p.createVisualShape(p.GEOM_BOX, halfExtents=[box_length/2, box_width/2, box_depth/2],
                                rgbaColor=[1,0,0,1])
boxId = p.createMultiBody(baseMass=0,
                          baseCollisionShapeIndex=geomBox,
                          baseVisualShapeIndex=visualBox,
                          basePosition=target_box_center,
                          baseOrientation=p.getQuaternionFromEuler([0,0,0]))

box2_length, box2_width, box2_depth = 0.2, 0.2, 0.6
target_box2_center = [1, 1, 0.6]
geomBox2 = p.createCollisionShape(p.GEOM_BOX, halfExtents=[box2_length/2, box2_width/2, box2_depth/2])
visualBox2 = p.createVisualShape(p.GEOM_BOX, halfExtents=[box2_length/2, box2_width/2, box2_depth/2],
                                rgbaColor=[1,0.5,0,1])
box2Id = p.createMultiBody(baseMass=0,
                          baseCollisionShapeIndex=geomBox2,
                          baseVisualShapeIndex=visualBox2,
                          basePosition=target_box2_center,
                          baseOrientation=p.getQuaternionFromEuler([0,0,0.5]))

# Place a blue sphere
sphere_radius = 0.25
sphere_position = [0.5, 1.5, 0.35]
geomSphere = p.createCollisionShape(p.GEOM_SPHERE, radius=sphere_radius)
visualSphere = p.createVisualShape(p.GEOM_SPHERE, radius=sphere_radius, rgbaColor=[0,0,1,1])
sphereId = p.createMultiBody(baseMass=0,
                             baseCollisionShapeIndex=geomSphere,
                             baseVisualShapeIndex=visualSphere,
                             basePosition=sphere_position,
                             baseOrientation=p.getQuaternionFromEuler([0,0,0]))

# Place a green square plate
plate_size = 2
plate_thickness = 0.1
plate_position = [1,1,plate_thickness/2]
geomPlate = p.createCollisionShape(p.GEOM_BOX, halfExtents=[plate_size/2, plate_size/2, plate_thickness/2])
visualPlate = p.createVisualShape(p.GEOM_BOX, halfExtents=[plate_size/2, plate_size/2, plate_thickness/2],
                                  rgbaColor=[0,1,0,1])
plateId = p.createMultiBody(baseMass=0,
                            baseCollisionShapeIndex=geomPlate,
                            baseVisualShapeIndex=visualPlate,
                            basePosition=plate_position,
                            baseOrientation=p.getQuaternionFromEuler([0,0,0]))


# Text description of the object to find
target_description = "The orange rectangular prism on the green plane"
text_features = encode_text(target_description)

joint_limits_lower = [-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973]
joint_limits_upper = [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]

roll_angles = []
tracking_errors = []


for ITER in range(100):
    p.stepSimulation()
    cameraPosition, cameraOrientation = robot.get_ee_position()
    rgb, depth = get_camera_img_float(cameraPosition, cameraOrientation)
    pil_img = Image.fromarray(rgb)

    # Generate candidate bounding boxes (for demonstration, assume we have a function)
    candidate_boxes = generate_candidate_boxes(rgb)

    best_similarity = -1
    best_box = None

    # Preprocess all candidate boxes first
    crops = []
    for box in candidate_boxes:
        x1, y1, x2, y2 = box
        cropped_img = pil_img.crop((x1, y1, x2, y2))
        crop_tensor = preprocess(cropped_img).unsqueeze(0)  # shape: (1, C, H, W)
        crops.append(crop_tensor)

    # Stack all crops into a single batch
    batch = torch.cat(crops, dim=0).to(device)  # shape: (N, C, H, W) where N = number of candidate boxes

    # Run model once on entire batch
    with torch.no_grad():
        image_features = model.encode_image(batch)  # shape: (N, D)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

    # Compute similarities for all boxes at once
    similarities = (image_features @ text_features.T).squeeze()  # shape: (N,)

    # Find best matching box
    best_idx = similarities.argmax().item()
    best_box = candidate_boxes[best_idx]
    best_similarity = similarities[best_idx].item()

    # Now best_box corresponds to the region where CLIP thinks the described object is
    # Extract the center of this box as your target pixel location
    (x1, y1, x2, y2) = best_box
    u_px = (x1 + x2) // 2
    v_px = (y1 + y2) // 2

    # Now feed (u_px, v_px) into your visual servoing steps:
    imageJacobian = getImageJacobian(u_px, v_px, depth, camera_focal_depth, camera_width, camera_height)
    delta_X, delta_Omega = findCameraControl(object_location_desired, [u_px, v_px], imageJacobian)
    delta_q = findJointControl(robot, delta_X, delta_Omega)
    
    current_q = robot.get_current_joint_angles()
    new_jointPositions = current_q + delta_q
    new_jointPositions = np.clip(new_jointPositions, joint_limits_lower, joint_limits_upper)
    robot.set_joint_position(new_jointPositions)
    
    
    tracking_errors.append(np.linalg.norm(object_location_desired - [u_px,v_px]))

    # Show debug info
    debug_img = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    cv2.circle(debug_img, (u_px, v_px), 5, (0,0,0), 2)
    cv2.circle(debug_img, (int(object_location_desired[0]), int(object_location_desired[1])), 5, (255,0,0), 2)
    cv2.rectangle(debug_img, (x1, y1), (x2, y2), (0, 0, 0), 2)  # Red rectangle
    

    cv2.imshow("Camera RGB", debug_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()
p.disconnect()


startThreads creating 1 threads.
starting thread 0
Initial EE pose: [0.05695421 0.05842742 1.11240143] [[ 0.7072639  -0.01201191  0.70684757]
 [-0.70694639 -0.00899466  0.70720993]
 [-0.00213709 -0.9998874  -0.01485337]]
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3070 Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.183.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.183.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3070 Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFun